<font size="+3" color='#053c96'><h2><center> Modeling</h2></center></font>
<figure>
<center><img src ="https://www.krasamo.com/wp-content/uploads/0730Building-ML-Models.jpg" width = "750" height = '600' alt="Classical Punk"/>
<font size="0" color='#053c96'><h4><center> Photo Credit: krasamo</h4></center></font>

<font size="+2" color='#053c96'><b> Contributor</b></font>  
<font size="+0" ><b> Umar Kabir</b></font>  

<a id='table-of-contents'></a>
[Table of Contents](#table-of-contents)

- [Modeling](#modeling)
  * [Data Transformation](#data-transformation)
  * [Model Selection](#model-selection)
  * [Model Training](#model-training)
  * [Model Evaluation](#model-evaluation)
  * [Hyperparameter Tuning](#hyperparameter-tuning)
- [Results](#results)
  * [Analysis Results](#analysis-results)
  * [Model Performance](#model-performance)
  * [Feature Importance](#feature-importance)
  * [Implications](#implications)
- [Conclusion](#conclusion)
  * [Summary](#summary)
  * [Limitations](#limitations)
  * [Recommendations](#recommendations)
- [References](#references)

<a id='importing-dependencies'></a>
<font size="+2" color='#053c96'><b> 1. Importing Libraries</b></font>  
[back to top](#table-of-contents)

In [1]:
import sys
# Insert the parent path relative to this notebook so we can import from the src folder.
sys.path.insert(0, "..")

from src.dependencies import *
from src.functions import *

<a id='data'></a>
<font size="+2" color='#053c96'><b> 2. Data</b></font>  
[back to top](#table-of-contents)  

The dataset was used for the well-known paper in genre classification "Musical genre classification of audio signals" by G. Tzanetakis and P. Cook in IEEE Transactions on Audio and Speech Processing 2002.

In [2]:
df = pd.read_csv("../data/processed_data.csv")

In [7]:
df = df.dropna()

<a id='modeling'></a>

<font size="+2" color='#053c96'><b> 4. Modeling</b></font>  
[back to top](#table-of-contents)

<a id='data-transformation'></a>
<font size="+0" color='green'><b> Data Transformation</b></font>  
[back to top](#table-of-contents)
<a id='modeling'></a>

<a id='model-selection'></a>

<font size="+1" color='#780404'><b> 4.1 Model Selection</b></font>  
[back to top](#table-of-contents)

<a id='model-training'></a>

<font size="+1" color='#780404'><b> 4.2 Model Training</b></font>  
[back to top](#table-of-contents)

<a id='model-evaluation'></a>

<font size="+1" color='#780404'><b> 4.3 Model Evaluation</b></font>  
[back to top](#table-of-contents)

<a id='hyperparameter-tuning'></a>
<font size="+1" color='#780404'><b> 4.4 Hyperparameter Tuning</b></font>  
[back to top](#table-of-contents)

<a id='results'></a>
<font size="+2" color='#053c96'><b> 5. Results</b></font>  
[back to top](#table-of-contents)

<a id='analysis-results'></a>

<font size="+1" color='#780404'><b> 5.1 Analysis Results</b></font>  
[back to top](#table-of-contents)

<a id='model-performance'></a>

<font size="+1" color='#780404'><b> 5.2 Model Performance</b></font>  
[back to top](#table-of-contents)

<a id='feature-importance'></a>

<font size="+1" color='#780404'><b> 5.3 Feature Importance</b></font>  
[back to top](#table-of-contents)

<a id='implications'></a>

<font size="+1" color='#780404'><b> 5.4 Implications</b></font>  
[back to top](#table-of-contents)

<a id='conclusion'></a>

<font size="+2" color='#053c96'><b> 6. Conclusion</b></font>  
[back to top](#table-of-contents)

<a id='summary'></a>

<font size="+1" color='#780404'><b> 6.1 Summary</b></font>  
[back to top](#table-of-contents)

<a id='limitations'></a>

<font size="+1" color='#780404'><b> 6.2 Limitations</b></font>  
[back to top](#table-of-contents)

<a id='recommendations'></a>

<font size="+1" color='#780404'><b> 6.3 Recommendations</b></font>  
[back to top](#table-of-contents)

<a id='references'></a>

<font size="+2" color='#053c96'><b> 7. References</b></font>  
[back to top](#table-of-contents)